In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [ ]:
import pickle 
with open('label_encoder.pkl','rb') as file:
 label_en=pickle.load(file)
with open('hot_encoder.pkl','rb') as file:
 onehot_en=pickle.load(file)
with open('standard_scaler.pkl','rb') as file:
 scaler=pickle.load(file)
model=load_model('model.h5')

In [7]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [15]:
input_dataf=pd.DataFrame([input_data])

hot_encoded = onehot_en.transform(input_dataf[['Geography']]).toarray()
hot_encoded

array([[1., 0., 0.]])

In [16]:


sep_names=onehot_en.get_feature_names_out(['Geography'])
input_dataf['Gender']=label_en.transform(input_dataf['Gender'])
input_dataf


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [18]:
new=pd.DataFrame(hot_encoded,columns=sep_names)
result=pd.concat([input_dataf.drop(['Geography'],axis=1),new],axis=1)
result


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [25]:
input_scaled=scaler.transform(result)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [26]:
## Predict churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 344ms/step


array([[0.53772813]], dtype=float32)

In [27]:
prediction_proba = prediction[0][0]

In [28]:
prediction_proba

0.53772813

In [29]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is likely to churn.
